In [1]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

2023-08-06 23:42:54 GM01X680 metapub.config[71004] WARNING NCBI_API_KEY was not set.


In [2]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [3]:
# www.sciencedirect.com
def www_sciencedirect_com(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Chrome(options=options)
            driver.get(url)
            time.sleep(3)
            # WebDriverWait(driver, 20).until(EC.element_to_be_clickable(By.XPATH, "//button[text()='Accept Cookies']")).click()
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # extract information from loaded webpage
    doi = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/div/div/div[2]/article/div[3]/a[1]").text.split("doi.org/")[1]
    try:
        doi = driver.find_element(By.CLASS_NAME, "anchor-text").text.split("doi.org/")[1]
    except:
        doi = np.nan
    pmid = np.nan
    pmcid = np.nan
    try:
        title = driver.find_element(By.TAG_NAME, 'h1').text
    except:
        title = np.nan
    try:
        abstract = driver.find_element(By.ID, "abstracts").find_element(By.TAG_NAME, "p").text
    except:
        abstract = np.nan
    keywords = np.nan
    try:
        intro = ""
        # Get the starting element
        elements = driver.find_element(By.LINK_TEXT, 'Introduction').find_element(By.TAG_NAME, 'section').find_elements(By.XPATH, 'p')
        for element in elements:
            intro = intro + element.text
    except:
        intro = np.nan
    # pdf_link = driver.find_element(By.XPATH, "//*[@id='download_article​']").get_attribute('href')
    try:
        pdf_link = driver.find_element(By.CLASS_NAME, "ViewPDF").find_element(By.XPATH, 'a').get_attribute('href')
    except:
        pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "introduction": intro,
        "pdf_link": pdf_link
    }

    return info
# --------------------start of test code--------------------
url = "https://www.sciencedirect.com/science/article/pii/030439409512056A"
info = www_sciencedirect_com(url)
print(info["doi"])
print(info["pmid"])
print(info["pmcid"])
print(info["title"])
print(info["abstract"])
print(info["keywords"])
print(info["introduction"])
print(info["pdf_link"])
# ---------------------end of test code---------------------

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div/div/div/div/div/div[2]/article/div[3]/a[1]"}
  (Session info: headless chrome=115.0.5790.170); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x55e622d2a613 <unknown>
#1 0x55e622a56537 <unknown>
#2 0x55e622a94f53 <unknown>
#3 0x55e622a95041 <unknown>
#4 0x55e622ad02a4 <unknown>
#5 0x55e622ab5a5d <unknown>
#6 0x55e622acdd09 <unknown>
#7 0x55e622ab5803 <unknown>
#8 0x55e622a8968b <unknown>
#9 0x55e622a8a42e <unknown>
#10 0x55e622cebc28 <unknown>
#11 0x55e622cefaf7 <unknown>
#12 0x55e622cfa01c <unknown>
#13 0x55e622cf0723 <unknown>
#14 0x55e622cbf457 <unknown>
#15 0x55e622d14628 <unknown>
#16 0x55e622d147b7 <unknown>
#17 0x55e622d23483 <unknown>
#18 0x7fe92d694b43 <unknown>
